In [1]:
import sys, os
sys.path.append(os.path.abspath(os.path.join('..')))  # if notebook is inside notebooks/
from src.scrapers import fortydash_pipeline as p
df = p.run_pipeline(start=2000, end=2025)  # writes to Dynasty/data/scraper/40yd_time.csv
df.head()

/Users/chasesiegel/Desktop/Comp_Sci/Capstone/Dynasty/src/scrapers/fortydash_pipeline.py:329: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(r.text)
/Users/chasesiegel/Desktop/Comp_Sci/Capstone/Dynasty/src/scrapers/fortydash_pipeline.py:329: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(r.text)


Wrote 2,905 rows to /Users/chasesiegel/Desktop/Comp_Sci/Capstone/Dynasty/data/scraper/40yd_time_2000_2025.csv


,player,year,pos,height_in,weight_lb,forty_s,is_estimate,estimate_method,confidence,source
0,Bill Burke,2000,QB,76.0,206.0,5.03,False,None,0.95,PFR:2000
1,Chad Pennington,2000,QB,75.0,229.0,4.81,False,None,0.95,PFR:2000
2,Chris Chaloupka,2000,QB,74.0,216.0,4.96,False,None,0.95,PFR:2000
3,Chris Redman,2000,QB,75.0,222.0,5.37,False,None,0.95,PFR:2000
4,Doug Johnson,2000,QB,74.0,226.0,4.95,False,None,0.95,PFR:2000
